In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
df=pd.read_csv(r"Data Engineer Assignment.csv")

In [3]:
df.dtypes

provider_id    object
detail         object
source         object
event_time     object
dtype: object

In [4]:
df['eventtime'] = df['event_time'].apply(lambda x : datetime.strptime(x[:-7],'%Y-%m-%d %H:%M:%S'))

In [5]:
df['date'] = df["event_time"].apply(lambda x:datetime.date(datetime.strptime(x[:-7],'%Y-%m-%d %H:%M:%S')))

In [6]:
df['time'] = df['event_time'].apply(lambda x : datetime.time(datetime.strptime(x[:-7],'%Y-%m-%d %H:%M:%S')))

In [7]:
df.head()

,provider_id,detail,source,event_time,eventtime,date,time
0,6873f0af240706d65514ec247c64f330,{u'mme': False},job_responded,2017-09-01 00:00:00.000000,2017-09-01 00:00:00,2017-09-01,00:00:00
1,a5a021d997269c4f1846903ff0f5d737,"{u'lat': 18.9810108, u'long': 72.8187476}",location_tracking,2017-09-01 00:00:02.000000,2017-09-01 00:00:02,2017-09-01,00:00:02
2,77af6d6d6db530c389ec1014d557a01e,{u'mme': False},decline_request,2017-09-01 00:00:05.000000,2017-09-01 00:00:05,2017-09-01,00:00:05
3,7ea218c2b9a6dada27e0d5bdf5f896de,{u'mme': False},decline_request,2017-09-01 00:00:12.000000,2017-09-01 00:00:12,2017-09-01,00:00:12
4,0d7a9d9c00231885b413175980a9beca,"{u'lat': 12.9959477, u'long': 77.6527175}",location_tracking,2017-09-01 00:00:15.000000,2017-09-01 00:00:15,2017-09-01,00:00:15


In [8]:
df1=df[df.time <datetime.time(datetime.strptime('1900-01-01 19:00:00','%Y-%m-%d %H:%M:%S'))]

In [10]:
df1.shape

(1613934, 7)

In [11]:
df1['offline']=df1["detail"].str.contains('False')


C:\Users\Vidya Sagar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1['online'] = (df1['detail'].str.contains('True') | df1['detail'].str.contains('Action on Job'))


C:\Users\Vidya Sagar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df1=df1.drop(df1[(df1.offline==False) & (df1.online==False)].index)


In [14]:
df1.loc[df1.offline == True, 'Status'] = 'offline'


In [15]:
df1.loc[df1.online == True, 'Status'] = 'online'

In [16]:
df1.shape

(765010, 10)

In [17]:
df1=df1.dropna()

In [22]:
df1['Status'].value_counts()

offline    520503
online     175164
Name: Status, dtype: int64

In [23]:
df1.shape

(695667, 11)

In [24]:
def get_status(df1):
    if df1['time'] <datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S')):
        return datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S'))
    elif df1['time'] >=datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S')):
        return df1['time']
    
df1['New_time'] = df1.apply(get_status, axis = 1)

In [28]:
df2 = df1.sort_values(['provider_id', 'date', "New_time"])


In [29]:
df2["offset_status"] = df2["Status"].shift(1)


In [30]:
df2["offset_time"] = df2["New_time"].shift(1)


In [31]:
df2["offset_date"] = df2["date"].shift(1)


In [32]:
df2["offset_pid"] = df2["provider_id"].shift(1)


In [33]:
df2.head()

,provider_id,detail,source,event_time,eventtime,date,time,offline,online,Status,New_time,offset_status,offset_time,offset_date,offset_pid
369418,00020da44b0a44e867de5f26c69886ac,{u'mme': False},decline_request,2017-09-07 00:30:46.000000,2017-09-07 00:30:46,2017-09-07,00:30:46,True,False,offline,08:00:00,NaN,NaN,NaN,NaN
53031,0002beae790876e71dfb99a904563c30,{u'mme': False},decline_request,2017-09-01 17:37:59.000000,2017-09-01 17:37:59,2017-09-01,17:37:59,True,False,offline,17:37:59,offline,08:00:00,2017-09-07,00020da44b0a44e867de5f26c69886ac
53035,0002beae790876e71dfb99a904563c30,{u'mme': False},decline_request,2017-09-01 17:38:05.000000,2017-09-01 17:38:05,2017-09-01,17:38:05,True,False,offline,17:38:05,offline,17:37:59,2017-09-01,0002beae790876e71dfb99a904563c30
36530,0002beae790876e71dfb99a904563c30,{u'mme': False},decline_request,2017-09-01 17:38:12.000000,2017-09-01 17:38:12,2017-09-01,17:38:12,True,False,offline,17:38:12,offline,17:38:05,2017-09-01,0002beae790876e71dfb99a904563c30
53042,0002beae790876e71dfb99a904563c30,{u'mme': False},decline_request,2017-09-01 17:38:18.000000,2017-09-01 17:38:18,2017-09-01,17:38:18,True,False,offline,17:38:18,offline,17:38:12,2017-09-01,0002beae790876e71dfb99a904563c30


In [34]:
def  online_secs (x):
    td_diff = timedelta(seconds=0)
    if x["provider_id"] == x["offset_pid"]:
        if x["date"] == x["offset_date"]:
            if x["Status"] == x["offset_status"] and x["Status"] == "offline":
                td_diff = timedelta(seconds=0) 
            elif x["Status"] == x["offset_status"] and x["Status"] == "online":
                strt_time = datetime.combine(x["offset_date"], x["offset_time"])
                end_time = datetime.combine(x["date"], x["New_time"])
                td_diff = end_time-strt_time
            elif x["Status"] != x["offset_status"] and x["Status"] == "online" and x["offset_status"] == "offline" :
                td_diff = timedelta(seconds=0) 
            elif x["Status"] != x["offset_status"] and x["Status"] == "offline" and x["offset_status"] == "online" :
                strt_time = datetime.combine(x["offset_date"], x["offset_time"])
                end_time = datetime.combine(x["date"], x["New_time"])
                td_diff = end_time-strt_time
    
    return td_diff.total_seconds() 


In [35]:
df2["seconds_online"] = df2.apply(online_secs, axis = 1)


In [36]:
df2[df2["seconds_online"] > 0.0]

,provider_id,detail,source,event_time,eventtime,date,time,offline,online,Status,New_time,offset_status,offset_time,offset_date,offset_pid,seconds_online
191410,0037dfffff8b03bbdf366a263735e84b,"{u'mme': False, u'requestID': ObjectId('59abe2...",fanout,2017-09-04 08:29:39.000000,2017-09-04 08:29:39,2017-09-04,08:29:39,True,False,offline,08:29:39,online,08:00:19,2017-09-04,0037dfffff8b03bbdf366a263735e84b,1760.0
224436,0037dfffff8b03bbdf366a263735e84b,"{u'mme': False, u'requestID': ObjectId('59acf8...",fanout,2017-09-04 12:23:24.000000,2017-09-04 12:23:24,2017-09-04,12:23:24,True,False,offline,12:23:24,online,09:06:12,2017-09-04,0037dfffff8b03bbdf366a263735e84b,11832.0
225210,0037dfffff8b03bbdf366a263735e84b,{u'mme': False},job_responded,2017-09-04 12:45:56.000000,2017-09-04 12:45:56,2017-09-04,12:45:56,True,False,offline,12:45:56,online,12:45:46,2017-09-04,0037dfffff8b03bbdf366a263735e84b,10.0
229356,0037dfffff8b03bbdf366a263735e84b,"{u'mme': True, u'Customer Request Id': u'59ace...",Provider Action on Job : end,2017-09-04 17:40:51.000000,2017-09-04 17:40:51,2017-09-04,17:40:51,False,True,online,17:40:51,online,17:38:31,2017-09-04,0037dfffff8b03bbdf366a263735e84b,140.0
258799,0037dfffff8b03bbdf366a263735e84b,"{u'mme': False, u'requestID': ObjectId('59ad22...",fanout,2017-09-05 09:29:21.000000,2017-09-05 09:29:21,2017-09-05,09:29:21,True,False,offline,09:29:21,online,09:00:13,2017-09-05,0037dfffff8b03bbdf366a263735e84b,1748.0
274403,0037dfffff8b03bbdf366a263735e84b,{u'Customer Request Id': u'59ad229719581e5d007...,Provider Action on Job : end,2017-09-05 13:15:55.000000,2017-09-05 13:15:55,2017-09-05,13:15:55,False,True,online,13:15:55,online,11:14:30,2017-09-05,0037dfffff8b03bbdf366a263735e84b,7285.0
299005,0037dfffff8b03bbdf366a263735e84b,"{u'mme': False, u'requestID': ObjectId('59ad71...",fanout,2017-09-05 15:29:36.000000,2017-09-05 15:29:36,2017-09-05,15:29:36,True,False,offline,15:29:36,online,13:15:55,2017-09-05,0037dfffff8b03bbdf366a263735e84b,8021.0
328911,0037dfffff8b03bbdf366a263735e84b,"{u'mme': True, u'Customer Request Id': u'59ae6...",Provider Action on Job : end,2017-09-06 11:59:07.000000,2017-09-06 11:59:07,2017-09-06,11:59:07,False,True,online,11:59:07,online,10:37:14,2017-09-06,0037dfffff8b03bbdf366a263735e84b,4913.0
334694,0037dfffff8b03bbdf366a263735e84b,"{u'mme': False, u'requestID': ObjectId('59afa3...",fanout,2017-09-06 12:58:44.000000,2017-09-06 12:58:44,2017-09-06,12:58:44,True,False,offline,12:58:44,online,11:59:07,2017-09-06,0037dfffff8b03bbdf366a263735e84b,3577.0
342610,0037dfffff8b03bbdf366a263735e84b,"{u'mme': True, u'Customer Request Id': u'59afa...",Provider Action on Job : end,2017-09-06 15:12:19.000000,2017-09-06 15:12:19,2017-09-06,15:12:19,False,True,online,15:12:19,online,14:57:04,2017-09-06,0037dfffff8b03bbdf366a263735e84b,915.0


In [37]:
df2["start_time"] = df2.New_time.apply(lambda x:int(str(x)[:2]))
df2["end_time"] = df2.New_time.apply(lambda x:int(str(x)[:2])+1)

In [38]:
df_final = df2.groupby(["provider_id", "date", "start_time", "end_time"])["seconds_online"].sum()

In [39]:
df_final1 = df_final.to_frame().reset_index()

In [40]:
df_final1[df_final1["seconds_online"] > 0.0]

,provider_id,date,start_time,end_time,seconds_online
261,0037dfffff8b03bbdf366a263735e84b,2017-09-04,8,9,1760.0
263,0037dfffff8b03bbdf366a263735e84b,2017-09-04,12,13,11842.0
264,0037dfffff8b03bbdf366a263735e84b,2017-09-04,17,18,140.0
265,0037dfffff8b03bbdf366a263735e84b,2017-09-05,9,10,1748.0
267,0037dfffff8b03bbdf366a263735e84b,2017-09-05,13,14,7285.0
268,0037dfffff8b03bbdf366a263735e84b,2017-09-05,15,16,8021.0
272,0037dfffff8b03bbdf366a263735e84b,2017-09-06,11,12,4913.0
273,0037dfffff8b03bbdf366a263735e84b,2017-09-06,12,13,3577.0
275,0037dfffff8b03bbdf366a263735e84b,2017-09-06,15,16,915.0
276,0037dfffff8b03bbdf366a263735e84b,2017-09-06,16,17,3958.0


In [41]:
 df_final1.sort_values(['provider_id', 'date', "start_time"])

,provider_id,date,start_time,end_time,seconds_online
0,00020da44b0a44e867de5f26c69886ac,2017-09-07,8,9,0.0
1,0002beae790876e71dfb99a904563c30,2017-09-01,17,18,0.0
2,0002beae790876e71dfb99a904563c30,2017-09-02,16,17,0.0
3,0002beae790876e71dfb99a904563c30,2017-09-03,11,12,0.0
4,0002beae790876e71dfb99a904563c30,2017-09-03,14,15,0.0
5,0002beae790876e71dfb99a904563c30,2017-09-05,14,15,0.0
6,0005d4e14f83f51f1bbbd28ea6d8b7e7,2017-09-01,8,9,0.0
7,0005d4e14f83f51f1bbbd28ea6d8b7e7,2017-09-01,15,16,0.0
8,0005d4e14f83f51f1bbbd28ea6d8b7e7,2017-09-01,16,17,0.0
9,0005d4e14f83f51f1bbbd28ea6d8b7e7,2017-09-02,8,9,0.0
